In [1]:
import torch
import torch.nn.functional as F
import utils
from importlib import reload
reload(utils)

<module 'utils' from '/home/amitaharoni/workspace/cv-sprint/models/utils.py'>

In [2]:
train_dl, valid_dl, test_dl = utils.make_dataloaders()

Files already downloaded and verified
Files already downloaded and verified


In [3]:
class TestNetwork(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.backbone = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3),
            torch.nn.ReLU(),
            torch.nn.AdaptiveAvgPool2d(output_size=1),
        )
        self.head = torch.nn.Linear(in_features=16, out_features=100)

    def forward(self, x: torch.Tensor):
        x = self.backbone(x)
        x = x.view(-1, 16)
        x = self.head(x)
        return x
    
model = TestNetwork().to(utils.device)

In [4]:
model

TestNetwork(
  (backbone): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): AdaptiveAvgPool2d(output_size=1)
  )
  (head): Linear(in_features=16, out_features=100, bias=True)
)

In [5]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())
utils.train_model(model, train_dl, valid_dl, loss_fn, optimizer, tensorboard_dir='../runs/00_silly')

100%|██████████| 157/157 [00:00<00:00, 316.47it/s]


Epoch 1/10: train_loss=4.5126, val_loss=4.3780


100%|██████████| 157/157 [00:00<00:00, 261.90it/s]


Epoch 2/10: train_loss=4.3360, val_loss=4.2915


100%|██████████| 157/157 [00:00<00:00, 278.53it/s]


...

100%|██████████| 157/157 [00:00<00:00, 290.80it/s]


Epoch 9/10: train_loss=4.1541, val_loss=4.1405


100%|██████████| 157/157 [00:00<00:00, 315.62it/s]

Epoch 10/10: train_loss=4.1423, val_loss=4.1303


TestNetwork(
  (backbone): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): AdaptiveAvgPool2d(output_size=1)
  )
  (head): Linear(in_features=16, out_features=100, bias=True)
)